<a href="https://colab.research.google.com/github/zs-nemecz/ExperimentalMethodsClass/blob/master/MatematikaiStat_Alapok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Null-hipotézis tesztelés

**Mekkora a valószínűsége annak, hogy ezt az adatot kaptam, azt feltételezve, hogy a null-hipotézis igaz?** 

Vagy: elég valószínűtlen ez az adat ahhoz, hogy elvessem a null-hipotézist?

Menete:

1. Az adatgyűjtés és adatelemzés előtt a kutatási hipotézis megfogalmazása: Mi a jóslatom?
2. A hipotézis szempontjából releváns adatok gyűjtése.
3. A null-hipotézis és az alternatív hipotézis megszabása.
4. Az alternatív hipotézisnek megfelelő modell illesztése az adathoz, és a teszt-statisztika kiszámítása (pl. t érték, F érték).
5. Mi a valószínűsége a kapott teszt statisztikának, ha a null-hipotézis igaz?
6. A 'statisztikai szignifikancia' megállapítása (p<0.05?)

(forrás: [Russel Poldrack - Statistical Thinking for the 21st Century](https://statsthinking21.github.io/statsthinking21-core-site/hypothesis-testing.html))

Gyakran használt tesztek: t-teszt, ANOVA és ezek nem-parametrikus változatai. 

# A megfelelő teszt kiválasztása

1. Mennyi független változóm van?
2. A kategorikus független változónak mennyi szintje van?
3. Függetlenek egymástól a kategórikus változó szintjei?
4. Megegyezik a variancia az egyes kategóriákban?
5. Normális eloszlásból származik a minta?

In [ ]:
# Könyvtárak és modulok impoortálása
import numpy as np # adatgenerálás, adattömbök
import scipy.stats as sp # statisztika
import matplotlib.pyplot as plt # ábrák
np.random.seed(123456) #random seed rögzítése, hogy ugyanazokat a mintákat kapjuk 

In [ ]:
plt.rcParams["figure.figsize"] = (15,7) # az alapértelmezett ábra nagyságának beállítása
plt.rcParams['lines.markersize'] = 10 # az alapértelmezett jelölő (pont/csillag az ábrán) méret beállítása

#0. Példák

##Egy-mintás t-próba

1. Mennyi kategorikus független változóm van? **1**
2. A kategorikus független változónak mennyi szintje van? **1**
3. Függetlenek egymástól a kategórikus változó szintjei? **-**
4. Megegyezik a variancia az egyes kategóriákban? **-**
5. Normális eloszlásból származik a minta? **igen**

Példa: Korábbi kutatások szerint a populációban a WAIS-IV IQ teszten elért átlag pontszám 100.
Egy középiskola érettségiző diákjaival felveszem az IQ tesztet, és megvizsgálom, az átlaguk eltér-e a populáció átlagtól.

###Adatgenerálás

In [ ]:
# az adatsor, ami az IQ teszten elért pontokat tartalmazza:
sample_school = np.random.normal(107,15, size=21).astype(int)
print(sample_school)
# a populáció átlag meghatározása:
population_mean = 100

###Hisztogram

In [ ]:
# nézzünk rá az IQ pontok eloszlására a mintában - milyennek tűnik az eloszlás?
plt.hist(sample_school)
plt.xlabel('IQ Scores')
plt.ylabel('Frequency')
# jelöljük a populáció átlagot piros csillaggal a hisztogramon:
plt.plot(population_mean, 1,'r*')
plt.show()# az ábra kirajzolása

###Eltér az eloszlás a normáltól?

Ehhez is el tudok végezni egy statisztikai tesztet, például a scipy.stats.normaltest() funkcióval. https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html

In [ ]:
# teszteljük, hogy az eloszlás szignifikánsan eltér-e a normál eloszlástól:
s_k, p = sp.normaltest(sample_school) # ez a teszt csak n>20 ad érvényes eredményt
if p < 0.05:
  print('Az eloszlás szignifikánsan eltér a normáltól')
else:
  print('Az eloszlás nem tér el a normáltól.')

### scipy.stats.ttest_1samp() - Egymintás t-teszt
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html


In [ ]:
t_val, p_val = sp.ttest_1samp(sample_school, population_mean)
print('Az egy-mintás t-próba eredménye:\n\n t-érték: {}\t p-érték: {} '.format(t_val, p_val))

### Az elméleti populáció és a minta összehasonlítása

In [ ]:
population = np.random.normal(100,15, size=1000).astype(int)
plt.hist(sample_school, alpha=0.7, density=True, label='class')
plt.hist(population, alpha=0.7,density=True, label='population')
plt.xlabel('IQ Scores')
plt.ylabel('Frequency')
plt.legend()
plt.show()

##Független mintás t-próba
1. Mennyi kategorikus független változóm van? **1**
2. A kategorikus független változónak mennyi szintje van? **2**
3. Függetlenek egymástól a kategórikus változó szintjei? **igen**
4. Megegyezik a variancia az egyes kategóriákban? **igen**
5. Normális eloszlásból származik a minta? **igen**

Példa: Egy klinikai (pl. depressziós) és egy kontroll-csoport munkamemória teljesítményét hasonlítom össze számterjedelem feladattal. A klinikai minta átlaga 6.8, szórása 2.2. A kontroll-csoport átlaga 7.6, a szórás 1.8. Mindkét csoportban 50-50 résztvevő van. 

###Adatgenerálás

In [ ]:
# a klinikai és kontoll minták meghatározása:
clinical_sample = np.random.normal(6.8,2.2, size=50).astype(int)
control_sample = np.random.normal(7.6,1.8, size=50).astype(int)
print(clinical_sample)
print(control_sample)

###Hisztogram

In [ ]:
# nézzünk rá a számterjedelem eloszlására a két mintában - milyennek tűnik az eloszlás?
# eltér a két minta szórása?
figure, axs = plt.subplots(1,3, sharex=True, sharey=True) # az ábra váza
axs[0].hist(clinical_sample, color='r')
axs[0].set_title('Clinical Sample')
axs[1].hist(control_sample, color='b')
axs[1].set_title('Control Sample')
axs[2].hist(clinical_sample, color='r', alpha = 0.7)
axs[2].hist(control_sample, color='b', alpha = 0.7)
axs[2].set_title('Both Samples')
for ax in axs:
  ax.set_xlabel('Digit Span')
  ax.set_ylabel('Frequency')
plt.show()

###Eltér az eloszlás a normáltól?

In [ ]:
# teszteljük, hogy az eloszlás szignifikánsan eltér-e a normál eloszlástól:
for sample, sample_name in zip([control_sample, clinical_sample], ['Kontroll', 'Klinikai']):
  s_k, p = sp.normaltest(sample) # ez a teszt csak n>20 ad érvényes eredményt
  if p < 0.05:
    print('{} minta: Az eloszlás szignifikánsan eltér a normáltól.'.format(sample_name))
  else:
    print('{} minta: Az eloszlás nem tér el a normáltól.'.format(sample_name))

###A varianciák megegyeznek?
Ehhez Levene tesztet végzünk. https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.levene.html

In [ ]:
w, p = sp.levene(control_sample, clinical_sample)
if p < 0.05:
  print('A Levene teszt alapján a minták két különböző varianciájú populációból származnak.')
else:
    print('A Levene teszt alapján a minták egyenlö varianciájú populációból származnak.')

###scipy.stats.ttest_ind() - Független mintás t-próba
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [ ]:
# ha minden rendben, végezzük el a független mintás t-próbát
t_val, p_val = sp.ttest_ind(clinical_sample,control_sample)
print('A független mintás t-próba eredménye:\n\n t-érték: {}\t p-érték: {} '.format(t_val, p_val))

In [ ]:
# nézzük meg a két minta értékeit boxplot ábrán is
plt.boxplot([control_sample, clinical_sample], labels=['Control', 'Clinical'])
plt.ylabel('Digit Span')
plt.show()

##Összetartozó mintás t-próba
1. Mennyi kategorikus független változóm van? **1**
2. A kategorikus független változónak mennyi szintje van? **2**
3. Függetlenek egymástól a kategórikus változó szintjei? **nem**
4. Megegyezik a variancia az egyes kategóriákban? **igen**
5. Normális eloszlásból származik a minta? **igen**

Példa: Babák nézési idejét vizsgáljuk, mikor olyan filmet néznek, amelyben a főhős a segítő karakter választja, és amikor az akadályozó karaktert választja. (Mint a Hamlin 2007). Ugyanazokat a babákat vizsgáljuk mindkét helyzetben.

###Adatgenerálás

In [ ]:
# a nézési idők meghatározása a két csoportban:
sample_helper = np.random.normal(10,5.8, size=16)
lookingtime_relationship = np.random.normal(1.2,0.4, size=sample_helper.shape)
sample_hinderer = sample_helper * lookingtime_relationship
print(sample_helper)
print(sample_hinderer)

###Az adatok személése

In [ ]:
# ábrák:
# 3 ábrtát szeretnék 
#-egyet a nézési idők összefüggésének
#-egyet a nézési idők hisztogramjának a 2 kondícióban
#-egyet a boxplotnak, amin mindkét kondíciót látom
figure, axs = plt.subplots(1,3)
axs[0].plot(sample_helper, sample_hinderer, 'ko')
axs[0].set_title('Relationship between looking times')
axs[0].set_ylabel('Looking time - Hinderer condition')
axs[0].set_xlabel('Looking time - Helper condition')
axs[1].hist(sample_helper, color='r', alpha=0.7)
axs[1].hist(sample_hinderer, color='b', alpha=0.7)
axs[1].set_title('Histograms of looking times per condition')
axs[1].set_xlabel('Looking Time (s)')
axs[1].set_ylabel('Frequency')
axs[2].boxplot([sample_helper, sample_hinderer], labels=['Helper','Hinderer'])
axs[1].set_title('Looking time difference between conditions')
axs[2].set_ylabel('Looking Time (s)')
plt.show()

###Eltér az eloszlás a normáltól?

In [ ]:
# teszteljük, hogy az eloszlás szignifikánsan eltér-e a normál eloszlástól:
# figyeljük meg, milyen figyelmeztetést kapunk!
for sample, sample_name in zip([sample_hinderer, sample_helper], ['Hátráltató', 'Segítő']):
  s_k, p = sp.normaltest(sample) # ez a teszt csak n>20 ad érvényes eredményt
  if p < 0.05:
    print('{} minta: Az eloszlás szignifikánsan eltér a normáltól.'.format(sample_name))
  else:
    print('{} minta: Az eloszlás nem tér el a normáltól.'.format(sample_name))

###A varianciák megegyeznek?
Ehhez Levene tesztet végzünk. https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.levene.html

In [ ]:
w, p = sp.levene(sample_helper, sample_hinderer)
if p < 0.05:
  print('A Levene teszt alapján a minták két különböző varianciájú populációból származnak.')
else:
    print('A Levene teszt alapján a minták egyenlő varianciájú populációból származnak.')

###scipy.stats.ttest_rel() - Összetartozó mintás t-próba
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [ ]:
# ha minden rendben, akkor elvégzem a t-tesztet:
t_val, p_val = sp.ttest_rel(sample_helper, sample_hinderer)
print('Az összetartozó mintás t-próba eredménye:\n\n t-érték: {}\t p-érték: {} '.format(t_val, p_val))

##Egyszempontos varianciaelemzés (One-Way ANOVA)
Példa: Három korcsoport életminőségét vizsgáljuk egy kérdőívvel.
1. Mennyi kategorikus független változóm van? **1**
2. A kategorikus független változónak hány szintje van? **2 vagy több**
3. Függetlenek egymástól a kategórikus változó szintjei? **igen**
4. Megegyezik a variancia az egyes kategóriákban? **igen**
5. Normális eloszlásból származik a minta? **igen**

###Adatgenerálás

In [ ]:
# az életminőségi értékek meghatározása a 3 csoportban:
sample_young_adult = np.random.normal(40,5.8, size=50)
sample_middle_age = np.random.normal(36,5.1, size=50)
sample_old_adult = np.random.normal(43,6.2, size=50)

###Az adatok személése

In [ ]:
# ábrázoljuk a 3 csoportban az életminőség eloszlását hisztogrammal
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)
axs[0].hist(sample_young_adult)
axs[0].set_title('Young adults')
axs[1].hist(sample_middle_age)
axs[1].set_title('Middle aged adults')
axs[2].hist(sample_old_adult)
axs[2].set_title('Older adults')
for ax in axs:
  ax.set_xlabel('Quality of life Scores')
  ax.set_ylabel('Frequency')
plt.show()

###Eltér az eloszlás a normáltól?

In [ ]:
# ellenőrizzük, hogy egyik csoportban sem tér el az életminőség eloszlás a normális eloszlástól
samples_and_names = zip([sample_young_adult, sample_middle_age, sample_old_adult], ['Fiatal', 'Középkorú', 'Idős'])
for sample, sample_name in samples_and_names:
  s_k, p = sp.normaltest(sample) # ez a teszt csak n>20 ad érvényes eredményt
  if p < 0.05:
    print('{} minta: Az eloszlás szignifikánsan eltér a normáltól.'.format(sample_name))
  else:
    print('{} minta: Az eloszlás nem tér el a normáltól.'.format(sample_name))

###A varianciák megegyeznek?
Ehhez Levene tesztet végzünk. https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.levene.html

In [ ]:
w, p = sp.levene(sample_young_adult, sample_middle_age, sample_old_adult)
if p < 0.05:
  print('A Levene teszt alapján a minták két különböző varianciájú populációból származnak.')
else:
    print('A Levene teszt alapján a minták egyenlő varianciájú populációból származnak.')

###Doboz és oszlopdiagramok

In [ ]:
# ábrázoljuk az adatokat oszlopdiagrammal, és dobozdiagrammal is.
# az oszlopdiagramhoz először ki kell számolnom az átlagokat, és meg kell
# határoznom, ezek hol legyenek az x tengelyen
mean_young = np.mean(sample_young_adult)
mean_middle = np.mean(sample_middle_age)
mean_old = np.mean(sample_old_adult)
# az átlagokat listába gyűjtöm a könnyebb kezelhetőség érdekében
means = [mean_young, mean_middle, mean_old] 

x = [1,2,3]

# elkészítem az ábra vázát, 1 sorban 2 al-ábrával:
figure, axs = plt.subplots(1,2, sharex=True, sharey=True)

# oszlopdiagram
axs[0].bar(x, means)
axs[0].set_title('Group Averages')

# dobozdiagram
axs[1].boxplot([sample_young_adult, sample_middle_age, sample_old_adult])
axs[1].set_title('Median Values and Deviation per Group')

# adjuk meg a csoportok címkéit, és az y tengely értékét
plt.xticks(x, ('Young', 'Middle Age', 'Older'))
for ax in axs:
  ax.set_ylabel('Quality of Life - Raw Points')
plt.show()

###scipy.stats.f_oneway() - Egyszempontos varianciaelemzés
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html

In [ ]:
# ha minden rendben, akkor elvégzem a varianciaelemzést:
f_val, p_val = sp.f_oneway(sample_young_adult, sample_middle_age, sample_old_adult)
print('Az egyszempontos varianciaelemzés eredménye:\n\n t-érték: {}\t p-érték: {:f} '.format(f_val, p_val))

### 0.1. Feladat
Ha szignifikáns eredményt kapok a varianciaelemzésen, abból még nem derül ki, melyik két csoport között mutatkozik különbség. Ehhez post-hoc tesztet kell végeznem. Az eddig bemutatott statiszkiai próbák közül melyik lenne erre alkalmas?

Végezd el a post-hoc összehasonlításokat a csoportok között:
1. fiatal és középkorú
2. középkorú és idős
3. fiatal és idős csoport

In [ ]:
# Young vs Middle Aged 


# Middle Aged vs Older


#Young vs Older

Vigyázz! Mivel 3 tesztet végzel, az eredményt korrigálni kell.
A Bonferroni korrekció szerint, az eredeti szignifikaciaszintet három próba elvégzése után harmadára kell csökkenteni.

Az eredeti szignifikanciaszint: p < 0.05

A tesztek száma: 3

Az új, Bonferroni korrigált szignifikancia szint: 0.01666.
A post-hoc próbák eredménye csak akkor 'szignifikáns', ha p értéke kisebb, mint 0.0166.

In [ ]:
n_tests = 3
p_level = 0.05
bonf_corrected_p_level = p_level/3
print(bonf_corrected_p_level)

# 1. Szimuláció: Mi történik, ha **nincs valódi különbség** a két populáció átlaga között?

Szimuláljunk egy esetet, amikor két populáció tagjainak egy tulajdonsága mentén valójában nincsen különbség a populációk között. 

Egy tudatlan kutatónak nincsen hozzáférése ehhez a populáció-szintű tudáshoz, így random mintavételezéssel és statisztikai próbákkal szertne többet kideríteni az adott tulajdonságról a populációkban.

Például: Egy idegen bolygón intelligens életet találtunk. A bolygón mindössze kétféle lény él, és egyedszámuk is korlátozott, 5000 - 5000 egyed él belőlük. A bolygó társadalmának vezére pontos adminisztrációt vezet minden egyedről, amely tartalmazza a repülési segességüket is. 

A földi embereknek azonban nincs hozzáfárése ehhez az adatbázishoz, és a földönkívüli politikai hatalom nem szándékozik megosztani ezt velünk. Mi azonban minél jobban meg szeretnénk ismerni ezt az idegen világot. Ahhoz kaptunk jogot, hogy korlátozott mintán mérjük a bolygó lakóit. A kutatásért azonban fizetni kell, minél nagyobb mintán dolgozunk, annál drágább a kutatás. 

A sebességet egy 10 km hosszú röppályán mérjük, és km/h egységben regisztráljuk.
Mindkét populációból random mintát veszünk, és független mintás t-próbával összvetjük a sebességüket.

##Határozzuk meg az elméleti populációkat.

In [ ]:
np.random.seed() # minden alaklommal új populáció/minta generálása
population_size = 5000 # tagok száma (populációnként)
population_mean = 100 # a populáció átlag az adott tulajdonság mentén
deviation = 20 # a szórás mindkét populációban

# populáció generálás, normál eloszlás szerint
A = np.random.normal(population_mean,deviation, size=population_size)
B = np.random.normal(population_mean,deviation, size=population_size)

In [ ]:
# nézzük meg az eloszlást a két populációban hisztogramokon
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)
axs[0].hist(A, color='r')
axs[0].set_title('populáció: A')
axs[1].hist(B, color='b')
axs[1].set_title('populáció: B')
axs[2].hist(A, color='r', alpha=0.7)
axs[2].hist(B, color='b', alpha=0.7)
axs[2].set_title('populáció: A és B')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()

##Vegyünk mintát a populációkból.

In [ ]:
# határozzuk meg a minta elemszámát
sample_size = 16
# válasszunk ki random tagokat a két populációból (egy tagot csak egyszer választunk be)
a_sample = np.random.choice(A,sample_size, replace=False)
b_sample = np.random.choice(B,sample_size, replace=False)

In [ ]:
# nézzük meg, hogyan néz ki az eloszlás a két mintában
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)
axs[0].hist(a_sample, color='y')
axs[0].set_title('minta: a')
axs[1].hist(b_sample, color='g')
axs[1].set_title('minta: b')
axs[2].hist(a_sample, color='y', alpha=0.7)
axs[2].hist(b_sample, color='g', alpha=0.7)
axs[2].set_title('minta: a és b')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()

In [ ]:
# ábrázoljuk, hogy a minta egyes pontjai hol helyezkednek el a hisztogram x tengelyén
# azaz a repülési sebesség mely értékét veszik fel, és ez hogyan viszonyul az eloszláshoz a 
# populációban
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)

# figyelem! itt a minta pontjainak y értékének nincs értelme! csak az 
# ábrázolás miatt van rá szükség
sample_y_val = np.random.normal(population_size/5, population_size/50, b_sample.shape)
axs[0].hist(A, color='r')
axs[0].plot(a_sample, sample_y_val, 'go')
axs[0].set_title('populáció: A')
axs[1].hist(B, color='b')
axs[1].plot(b_sample, sample_y_val, 'yo')
axs[1].set_title('populáció: B')
axs[2].hist(B, color='b', alpha=0.7)
axs[2].hist(A, color='r', alpha=0.7)
axs[2].plot(a_sample, sample_y_val, 'go')
axs[2].plot(b_sample, sample_y_val, 'yo')
axs[2].set_title('populáció: A és B')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()


##Hasonlítsuk össze a mintákat független mintás t-próbával.

In [ ]:
t_val, p_val = sp.ttest_ind(a_sample,b_sample)
print('A független mintás t-próba eredménye:\n\n t-érték: {}\t p-érték: {} '.format(t_val, p_val))

##Mi történik, ha többször veszek mintát, és ezeket újra és újra tesztelem? Hogyan befolyásolja a minta nagysága és a tesztelés száma az eredményeket?

Például: A földönkívüliek repülési sebessége jelenleg nagyon népszerű kutatási téma. 100 labor is dolgozik ezen a kérdésen. Mennyi labor talál eltérést a két lény között repülési sebességben? 

In [ ]:
t_vals = []
p_vals = []
n_tests = 100
sample_size = 16
sig_counter = 0
for test in range(n_tests):
  a_sample = np.random.choice(A,sample_size, replace=False)
  b_sample = np.random.choice(B,sample_size, replace=False)
  t_val, p_val = sp.ttest_ind(a_sample,b_sample)
  t_vals.append(t_val)
  p_vals.append(p_val)
  if p_val < 0.05:
    sig_counter = sig_counter + 1

print('From {} tests, {} were signitficant, even though there is no real difference in the population mean.'.format(n_tests, sig_counter))

In [ ]:
#írjunk egy funkciót, ami meghatározott mintaelemszámmal n-szer lefuttatja a tesztet
def check_my_ttest(population_A, population_B, n_tests, sample_size):
  t_vals = []
  p_vals = []
  sig_counter = 0
  for test in range(n_tests):
    a_sample = np.random.choice(population_A,sample_size, replace=False)
    b_sample = np.random.choice(population_B,sample_size, replace=False)
    t_val, p_val = sp.ttest_ind(a_sample,b_sample)
    t_vals.append(t_val)
    p_vals.append(p_val)
    if p_val < 0.05:
      sig_counter = sig_counter + 1

  print('From {} tests, {} were signitficant.'.format(n_tests, sig_counter))

  return (t_vals, p_vals)

In [ ]:
t_vals, p_vals = check_my_ttest(A,B,100,10)

### 1.1.1. Feladat
Mi történik, ha többször végzed el ezt a szimulációt (100 teszt)? 100 tesztből átlagosan mennyi lesz szignifikáns, ha 10 szimulációt végzel? 

In [ ]:
# az átlagszámításhoz érdemes újradefiniálni a funkciót, hogy ne a t és p értékekkel
# térjen vissza, hanem a szignifikáns eredmények számával

def check_n_significant(population_A, population_B, n_tests, sample_size): #adjunk új nevet a funkciónak
  sig_counter = 0
  for test in range(n_tests):
    a_sample = np.random.choice(population_A,sample_size, replace=False)
    b_sample = np.random.choice(population_B,sample_size, replace=False)
    t_val, p_val = sp.ttest_ind(a_sample,b_sample)
    if p_val < 0.05:
      sig_counter = sig_counter + 1

  print('From {} tests, {} were signitficant.'.format(n_tests, sig_counter))

  return sig_counter

In [ ]:
# csináljunk egy for loop-ot, ami 10-szer lefuttatja a tesztet
n_simulations = 10
# legyen egy változó, amelyben az összes szimuláció eredményét összevonom
n_all_sig = 0
for i in range(n_simulations):
  # végezzük el az adott szimulációt
  result = check_n_significant(A, B, 100, 100) 
  # adjuk hozzá az értéket az eddigi szimulációk eredményéhez
  n_all_sig = n_all_sig + result

average_n_sig = n_all_sig / n_simulations
print('{} szimulációval átlagosan {} szignifikáns eredményt kaptunk.'.format(n_simulations,average_n_sig))

#2. Szimuláció: Mi történik, ha **van különbség** a két populáció átlaga között?

## 2.1: Ha a populációk átlagukban különböznek, de a szórásuk megegyezik:

In [ ]:
# határozzuk meg a két populációt, amely az adott tulajdonság mentén különbözik
population_size = 5000
population_mean = 100 # átlag a két populációt összevonva
deviation = 15 # szórás az populációkban
population_mean_difference = deviation # a különbség mértéke a populációkat összevonó átlagtól

A = np.random.normal(population_mean-population_mean_difference, deviation, size=population_size)
B = np.random.normal(population_mean+population_mean_difference, deviation, size=population_size)

In [ ]:
# nézzük meg, hogyan néz ki az eloszlás a két mintában
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)
axs[0].hist(A, color='r')
axs[0].set_title('populáció: A')
axs[1].hist(B, color='b')
axs[1].set_title('populáció: B')
axs[2].hist(A, color='r', alpha=0.7)
axs[2].hist(B, color='b', alpha=0.7)
axs[2].set_title('populáció: A és B')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()

In [ ]:
# határozzuk meg a minta elemszámát
sample_size = 16
# válasszunk ki random tagokat a két populációból (egy tagot csak egyszer választunk be)
a_sample = np.random.choice(A,sample_size, replace=False)
b_sample = np.random.choice(B,sample_size, replace=False)

In [ ]:
# nézzük meg, hogyan néz ki az eloszlás a két mintában
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)
axs[0].hist(a_sample, color='y')
axs[0].set_title('minta: a')
axs[1].hist(b_sample, color='g')
axs[1].set_title('minta: b')
axs[2].hist(a_sample, color='y', alpha=0.7)
axs[2].hist(b_sample, color='g', alpha=0.7)
axs[2].set_title('minta: a és b')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()

In [ ]:
# ábrázoljuk, hogy a minta egyes pontjai hol helyezkednek el a hisztogram x tengelyén
# azaz a repülési sebesség mely értékét veszik fel, és ez hogyan viszonyul az eloszláshoz a 
# populációban
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)

# figyelem! itt a minta pontjainak y értékének nincs értelme! csak az 
# ábrázolás miatt van rá szükség
sample_y_val = np.random.normal(population_size/5, population_size/50, b_sample.shape)
axs[0].hist(A, color='r')
axs[0].plot(a_sample, sample_y_val, 'go')
axs[0].set_title('populáció: A')
axs[1].hist(B, color='b')
axs[1].plot(b_sample, sample_y_val, 'yo')
axs[1].set_title('populáció: B')
axs[2].hist(B, color='b', alpha=0.7)
axs[2].hist(A, color='r', alpha=0.7)
axs[2].plot(a_sample, sample_y_val, 'go')
axs[2].plot(b_sample, sample_y_val, 'yo')
axs[2].set_title('populáció: A és B')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()

###Hasonlítsuk össze a mintákat független mintás t-próbával.

In [ ]:
t_val, p_val = sp.ttest_ind(a_sample,b_sample)
print('A független mintás t-próba eredménye:\n\n t-érték: {}\t p-érték: {:f} '.format(t_val, p_val))

###2.1.1. Feladat
Ábrázold dobozdiagrammal a fenti minták közötti különbséget.

###2.1.2. Feladat
a) Készíts egy szimulációt, amelyben a 100-szor veszel random mintát a populációból és független mintás t-próbát végzel a mintán. 100-ból hányszor kapsz szignifikáns eredményt? 

b) Mi történik, ha többször végzed el ezt a szimulációt (100 teszt)? 100 tesztből átlagosan mennyi lesz szignifikáns, ha 10 szimulációt végzel? 

c) Változtasd meg a különbség mértékét a populációk között!
Hogyan változik a szimulációk eredménye?

In [ ]:
#@title
# a) emlékezzz, hogy erre a kérdésre már csináltunk egy funkciót!
t_vals, p_vals = ????

In [ ]:
# b)

In [ ]:
# c)

##2.2: Ha a populációk az átlagukban különböznek, és a szórásuk is különbözik:

In [ ]:
population_size = 5000 # elemszám
population_mean = 100 # populációk összevont átlaga
deviation_A = 15 # szórás az A populációban
devaition_B = deviation_A * 4 # szórás a B populációban
population_mean_difference = deviation # összevont átlagtól való eltérés

A = np.random.normal(population_mean-population_mean_difference, deviation_A, size=population_size)
B = np.random.normal(population_mean+population_mean_difference, devaition_B, size=population_size)

In [ ]:
# nézzük meg, hogyan néz ki az eloszlás a két mintában
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)
axs[0].hist(A, color='r')
axs[0].set_title('populáció: A')
axs[1].hist(B, color='b')
axs[1].set_title('populáció: B')
axs[2].hist(A, color='r', alpha=0.7)
axs[2].hist(B, color='b', alpha=0.7)
axs[2].set_title('populáció: A és B')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()

In [ ]:
# határozzuk meg a minta elemszámát
sample_size = 16
# válasszunk ki random tagokat a két populációból (egy tagot csak egyszer választunk be)
a_sample = np.random.choice(A,sample_size, replace=False)
b_sample = np.random.choice(B,sample_size, replace=False)

###2.2.1 Feladat
Ábrázold a minták eloszlását hisztogramon.

In [ ]:
# ábrázoljuk, hogy a minta egyes pontjai hol helyezkednek el a hisztogram x tengelyén
# azaz a repülési sebesség mely értékét veszik fel, és ez hogyan viszonyul az eloszláshoz a 
# populációban
figure, axs = plt.subplots(1,3, sharex=True, sharey=True)

# figyelem! itt a minta pontjainak y értékének nincs értelme! csak az 
# ábrázolás miatt van rá szükség
sample_y_val = np.random.normal(population_size/5, population_size/50, b_sample.shape)
axs[0].hist(A, color='r')
axs[0].plot(a_sample, sample_y_val, 'go')
axs[0].set_title('populáció: A')
axs[1].hist(B, color='b')
axs[1].plot(b_sample, sample_y_val, 'yo')
axs[1].set_title('populáció: B')
axs[2].hist(B, color='b', alpha=0.7)
axs[2].hist(A, color='r', alpha=0.7)
axs[2].plot(a_sample, sample_y_val, 'go')
axs[2].plot(b_sample, sample_y_val, 'yo')
axs[2].set_title('populáció: A és B')
for ax in axs:
  ax.set_xlabel('Sebesség (km/h)')
  ax.set_ylabel('Gyakoriság')
plt.show()

###2.2.2. Feladat 
Végezz el a fenti mintán egy független mintás t-róbát! Érvényes ez az eredmény?
(Teljesülnek a t-próba követelményei?)

In [ ]:
t_val, p_val = ????
print('A független mintás t-próba eredménye:\n\n t-érték: {}\t p-érték: {:f} '.format(t_val, p_val))

###2.2.3. Feladat
a) Készíts egy szimulációt, amelyben a 100-szor veszel random mintát a populációból és független mintás t-próbát végzel a mintán. 100-ból hányszor kapsz szignifikáns eredményt? 

b) Mi történik, ha többször végzed el ezt a szimulációt (100 teszt)? 100 tesztből átlagosan mennyi lesz szignifikáns, ha 10 szimulációt végzel? 

c) Változtasd meg a különbség mértékét a populációk között!
Hogyan változik a szimulációk eredménye?